In [1]:
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 18.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.6 MB/s

In [2]:
!unzip tts_outputs.zip

Archive:  tts_outputs.zip
   creating: tts_outputs/
  inflating: tts_outputs/output_1.wav  
  inflating: tts_outputs/output_10.wav  
  inflating: tts_outputs/output_2.wav  
  inflating: tts_outputs/output_3.wav  
  inflating: tts_outputs/output_4.wav  
  inflating: tts_outputs/output_5.wav  
  inflating: tts_outputs/output_6.wav  
  inflating: tts_outputs/output_7.wav  
  inflating: tts_outputs/output_8.wav  
  inflating: tts_outputs/output_9.wav  


In [3]:
import os
import torch
import torchaudio
from transformers import WhisperForConditionalGeneration, WhisperProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "Huseyin/whisper-large-v3-turkish-finetuned"
model = WhisperForConditionalGeneration.from_pretrained(model_id).to(device)
processor = WhisperProcessor.from_pretrained(model_id)
input_dir = "tts_outputs"
output_dir = "stt_outputs"
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith(".wav"):
        path = os.path.join(input_dir, filename)
        waveform, sr = torchaudio.load(path)
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
            waveform = resampler(waveform)
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        input_features = processor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt").input_features.to(device)


        predicted_ids = model.generate(input_features)
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

        output_file = os.path.join(output_dir, filename.replace(".wav", ".txt"))
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(transcription)

print("🚀 Tüm dosyalar başarıyla transkribe edildi.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

`generation_config` default values have been modified to match model-specific defaults: {'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these values explicitly.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> to see related `.generate()` flags.


🚀 Tüm dosyalar başarıyla transkribe edildi.


In [4]:
!zip stt_outputs.zip stt_outputs/*

  adding: stt_outputs/output_10.txt (deflated 3%)
  adding: stt_outputs/output_1.txt (deflated 30%)
  adding: stt_outputs/output_2.txt (stored 0%)
  adding: stt_outputs/output_3.txt (stored 0%)
  adding: stt_outputs/output_4.txt (stored 0%)
  adding: stt_outputs/output_5.txt (stored 0%)
  adding: stt_outputs/output_6.txt (stored 0%)
  adding: stt_outputs/output_7.txt (stored 0%)
  adding: stt_outputs/output_8.txt (stored 0%)
  adding: stt_outputs/output_9.txt (stored 0%)
